In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

Target:

    Add dropout layers to minimize overfitting. 10% dropout to every convolution layer
    Batch normalization layers to normalize the input data. Added batch normalization after max pooling
    Max epochs = 15

Result:

    Parameters : 
      1st run : 49,578
      2nd run : 17,930


Analysis:

    Tried dropout = 10% and 20%, yet there is lot of overfitting.
    Reduced the channel size to 16 in multiple layers and kept dropout to 10%. This has solved the overfitting problem. The model accuracy is gradually increasing, yet there is no consistency
    


In [0]:
# Original network with less parameters, so that training is also faster
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop1 = nn.Dropout(0.1)  
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.bnm2d1 = nn.BatchNorm2d(32) 
        self.conv3 = nn.Conv2d(32, 16, 3, padding=1)
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.bnm2d2 = nn.BatchNorm2d(16) 
        self.conv5 = nn.Conv2d(16, 16, 3)
        self.conv6 = nn.Conv2d(16, 16, 3)
        self.conv7 = nn.Conv2d(16, 10, 3)

    def forward(self, x):
        x = self.bnm2d1(self.pool1(F.relu(self.drop1(self.conv2(F.relu(self.drop1(self.conv1(x))))))))
        x = self.bnm2d2(self.pool2(F.relu(self.drop1(self.conv4(F.relu(self.drop1(self.conv3(x))))))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
           Dropout-2           [-1, 16, 28, 28]               0
            Conv2d-3           [-1, 32, 28, 28]           4,640
           Dropout-4           [-1, 32, 28, 28]               0
         MaxPool2d-5           [-1, 32, 14, 14]               0
       BatchNorm2d-6           [-1, 32, 14, 14]              64
            Conv2d-7           [-1, 16, 14, 14]           4,624
           Dropout-8           [-1, 16, 14, 14]               0
            Conv2d-9           [-1, 16, 14, 14]           2,320
          Dropout-10           [-1, 16, 14, 14]               0
        MaxPool2d-11             [-1, 16, 7, 7]               0
      BatchNorm2d-12             [-1, 16, 7, 7]              32
           Conv2d-13             [-1, 16, 5, 5]           2,320
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
train_acc = []
test_acc = []
train_losses = []
test_losses = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0

    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        y_pred = model(data)

        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        # pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
    # def __init__(self):
    #     super(Net, self).__init__()
    #     self.drop1 = nn.Dropout(0.1)  
    #     self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
    #     self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
    #     self.pool1 = nn.MaxPool2d(2, 2)
    #     self.bnm2d1 = nn.BatchNorm2d(32) 
    #     self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
    #     self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
    #     self.pool2 = nn.MaxPool2d(2, 2)
    #     # self.bnm2d2 = nn.BatchNorm2d(32) 
    #     self.conv5 = nn.Conv2d(32, 32, 3)
    #     self.conv6 = nn.Conv2d(32, 32, 3)
    #     self.conv7 = nn.Conv2d(32, 10, 3)

    # def forward(self, x):
    #     x = self.bnm2d1(self.pool1(F.relu(self.drop1(self.conv2(F.relu(self.drop1(self.conv1(x))))))))
    #     x = self.bnm2d1(self.pool2(F.relu(self.drop1(self.conv4(F.relu(self.drop1(self.conv3(x))))))))
    #     x = F.relu(self.drop1(self.conv6(F.relu(self.conv5(x)))))
    #     x = self.conv7(x)
    #     x = x.view(-1, 10)
    #     return F.log_softmax(x)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.0966009721159935 batch_id=468 Accuracy=92.95: 100%|██████████| 469/469 [00:17<00:00, 26.96it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1097, Accuracy: 9679/10000 (96.79%)



loss=0.03172171488404274 batch_id=468 Accuracy=98.38: 100%|██████████| 469/469 [00:17<00:00, 27.19it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1146, Accuracy: 9642/10000 (96.42%)



loss=0.002573564648628235 batch_id=468 Accuracy=98.82: 100%|██████████| 469/469 [00:17<00:00, 27.09it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0488, Accuracy: 9843/10000 (98.43%)



loss=0.00031905373907648027 batch_id=468 Accuracy=99.62: 100%|██████████| 469/469 [00:16<00:00, 27.68it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0561, Accuracy: 9824/10000 (98.24%)



loss=0.0019700725097209215 batch_id=468 Accuracy=99.69: 100%|██████████| 469/469 [00:16<00:00, 27.64it/s]



Test set: Average loss: 0.0753, Accuracy: 9755/10000 (97.55%)



In [0]:
    # def __init__(self):
    #     super(Net, self).__init__()
    #     self.drop1 = nn.Dropout(0.2)  
    #     self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
    #     self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
    #     self.pool1 = nn.MaxPool2d(2, 2)
    #     self.bnm2d1 = nn.BatchNorm2d(32) 
    #     self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
    #     self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
    #     self.pool2 = nn.MaxPool2d(2, 2)
    #     # self.bnm2d2 = nn.BatchNorm2d(32) 
    #     self.conv5 = nn.Conv2d(32, 32, 3)
    #     self.conv6 = nn.Conv2d(32, 32, 3)
    #     self.conv7 = nn.Conv2d(32, 10, 3)

    # def forward(self, x):
    #     x = self.bnm2d1(self.pool1(F.relu(self.drop1(self.conv2(F.relu(self.drop1(self.conv1(x))))))))
    #     x = self.bnm2d1(self.pool2(F.relu(self.drop1(self.conv4(F.relu(self.drop1(self.conv3(x))))))))
    #     x = F.relu(self.drop1(self.conv6(F.relu(self.conv5(x)))))
    #     x = self.conv7(x)
    #     x = x.view(-1, 10)
    #     return F.log_softmax(x)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.07161596417427063 batch_id=468 Accuracy=91.20: 100%|██████████| 469/469 [00:17<00:00, 27.15it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1017, Accuracy: 9775/10000 (97.75%)



loss=0.03723248466849327 batch_id=468 Accuracy=98.01: 100%|██████████| 469/469 [00:17<00:00, 27.11it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0918, Accuracy: 9772/10000 (97.72%)



loss=0.013196036219596863 batch_id=468 Accuracy=98.53: 100%|██████████| 469/469 [00:17<00:00, 27.09it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0914, Accuracy: 9703/10000 (97.03%)



loss=0.0009019821882247925 batch_id=468 Accuracy=99.37: 100%|██████████| 469/469 [00:17<00:00, 27.00it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0968, Accuracy: 9720/10000 (97.20%)



loss=0.0041167535819113255 batch_id=468 Accuracy=99.46: 100%|██████████| 469/469 [00:17<00:00, 27.22it/s]



Test set: Average loss: 0.1502, Accuracy: 9525/10000 (95.25%)



In [0]:
# def __init__(self):
#     super(Net, self).__init__()
#     self.drop1 = nn.Dropout(0.2)  
#     self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
#     self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
#     self.pool1 = nn.MaxPool2d(2, 2)
#     self.bnm2d1 = nn.BatchNorm2d(32) 
#     self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
#     self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
#     self.pool2 = nn.MaxPool2d(2, 2)
#     # self.bnm2d2 = nn.BatchNorm2d(32) 
#     self.conv5 = nn.Conv2d(32, 32, 3)
#     self.conv6 = nn.Conv2d(32, 32, 3)
#     self.conv7 = nn.Conv2d(32, 10, 3)

# def forward(self, x):
#     x = self.bnm2d1(self.pool1(F.relu(self.drop1(self.bnm2d1(self.conv2(F.relu(self.drop1(self.bnm2d1(self.conv1(x))))))))))
#     x = self.bnm2d1(self.pool2(F.relu(self.drop1(self.bnm2d1(self.conv4(F.relu(self.drop1(self.bnm2d1(self.conv3(x))))))))))
#     x = F.relu(self.conv6(F.relu(self.conv5(x)))))
#     x = self.conv7(x)
#     x = x.view(-1, 10)
#     return F.log_softmax(x)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)    

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.07407665997743607 batch_id=468 Accuracy=92.17: 100%|██████████| 469/469 [00:17<00:00, 26.74it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.2713, Accuracy: 1599/10000 (15.99%)



loss=0.025794098153710365 batch_id=468 Accuracy=98.36: 100%|██████████| 469/469 [00:17<00:00, 26.73it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.1116, Accuracy: 2091/10000 (20.91%)



loss=0.027293339371681213 batch_id=468 Accuracy=98.72: 100%|██████████| 469/469 [00:17<00:00, 26.84it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 1.7374, Accuracy: 3019/10000 (30.19%)



loss=0.0010995169868692756 batch_id=468 Accuracy=99.53: 100%|██████████| 469/469 [00:17<00:00, 26.68it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.9795, Accuracy: 2468/10000 (24.68%)



loss=0.0022342156153172255 batch_id=468 Accuracy=99.53: 100%|██████████| 469/469 [00:17<00:00, 26.57it/s]



Test set: Average loss: 1.9968, Accuracy: 2287/10000 (22.87%)



In [0]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.drop1 = nn.Dropout(0.2)  
#         self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
#         self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
#         self.pool1 = nn.MaxPool2d(2, 2)
#         self.bnm2d1 = nn.BatchNorm2d(32) 
#         self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
#         self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
#         self.pool2 = nn.MaxPool2d(2, 2)
#         # self.bnm2d2 = nn.BatchNorm2d(32) 
#         self.conv5 = nn.Conv2d(32, 32, 3)
#         self.conv6 = nn.Conv2d(32, 32, 3)
#         self.conv7 = nn.Conv2d(32, 10, 3)

#     def forward(self, x):
#         x = self.bnm2d1(self.pool1(F.relu(self.drop1(self.conv2(F.relu(self.drop1(self.conv1(x))))))))
#         x = self.bnm2d1(self.pool2(F.relu(self.drop1(self.conv4(F.relu(self.drop1(self.conv3(x))))))))
#         x = F.relu(self.conv6(F.relu(self.conv5(x))))
#         x = self.conv7(x)
#         x = x.view(-1, 10)
#         return F.log_softmax(x)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)  

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.05745892599225044 batch_id=468 Accuracy=91.97: 100%|██████████| 469/469 [00:16<00:00, 27.76it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1702, Accuracy: 9624/10000 (96.24%)



loss=0.03604716435074806 batch_id=468 Accuracy=98.29: 100%|██████████| 469/469 [00:16<00:00, 27.61it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1485, Accuracy: 9696/10000 (96.96%)



loss=0.007841969840228558 batch_id=468 Accuracy=98.73: 100%|██████████| 469/469 [00:16<00:00, 27.85it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0707, Accuracy: 9813/10000 (98.13%)



loss=0.0006015797262080014 batch_id=468 Accuracy=99.57: 100%|██████████| 469/469 [00:16<00:00, 27.77it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0836, Accuracy: 9775/10000 (97.75%)



loss=0.0006982485647313297 batch_id=468 Accuracy=99.60: 100%|██████████| 469/469 [00:16<00:00, 27.77it/s]



Test set: Average loss: 0.1120, Accuracy: 9645/10000 (96.45%)



In [0]:
# def __init__(self):
#     super(Net, self).__init__()
#     self.drop1 = nn.Dropout(0.1)  
#     self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -? OUtput? RF
#     self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
#     self.pool1 = nn.MaxPool2d(2, 2)
#     self.bnm2d1 = nn.BatchNorm2d(32) 
#     self.conv3 = nn.Conv2d(32, 16, 3, padding=1)
#     self.conv4 = nn.Conv2d(16, 16, 3, padding=1)
#     self.pool2 = nn.MaxPool2d(2, 2)
#     self.bnm2d2 = nn.BatchNorm2d(16) 
#     self.conv5 = nn.Conv2d(16, 16, 3)
#     self.conv6 = nn.Conv2d(16, 16, 3)
#     self.conv7 = nn.Conv2d(16, 10, 3)

# def forward(self, x):
#     x = self.bnm2d1(self.pool1(F.relu(self.drop1(self.conv2(F.relu(self.drop1(self.conv1(x))))))))
#     x = self.bnm2d2(self.pool2(F.relu(self.drop1(self.conv4(F.relu(self.drop1(self.conv3(x))))))))
#     x = F.relu(self.conv6(F.relu(self.conv5(x))))
#     x = self.conv7(x)
#     x = x.view(-1, 10)
#     return F.log_softmax(x)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)  

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.117683045566082 batch_id=468 Accuracy=89.54: 100%|██████████| 469/469 [00:16<00:00, 28.03it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0943, Accuracy: 9785/10000 (97.85%)



loss=0.32682308554649353 batch_id=468 Accuracy=97.69: 100%|██████████| 469/469 [00:16<00:00, 28.03it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0607, Accuracy: 9849/10000 (98.49%)



loss=0.07210642099380493 batch_id=468 Accuracy=98.20: 100%|██████████| 469/469 [00:16<00:00, 28.08it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0209, Accuracy: 9933/10000 (99.33%)



loss=0.012974818237125874 batch_id=468 Accuracy=99.28: 100%|██████████| 469/469 [00:16<00:00, 27.89it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0233, Accuracy: 9926/10000 (99.26%)



loss=0.03186799958348274 batch_id=468 Accuracy=99.23: 100%|██████████| 469/469 [00:16<00:00, 27.88it/s]



Test set: Average loss: 0.0217, Accuracy: 9939/10000 (99.39%)

